In [1]:
#dependencies
import pandas as pd
import requests

In [2]:
#read in geoJSON and state codes
states = pd.read_json('./static/geoJson/us_states.json')
state_codes = pd.read_csv('./CSVs/state_codes.csv')

In [3]:
#Drop territories
state_list = state_codes[['State/District','Postal Code']][:51]

counter = 0

#Add state codes to geoJSON for easier lookups
for state in state_list.iterrows():
    state_code = state_list['Postal Code'][counter]
    states['features'][counter]['properties']['state_code'] = state_code
    counter = counter + 1
    
#PR gets a code so the program doesn't bomb later
states['features'][51]['properties']['state_code'] = 'PR'

In [4]:
#JSON requests to api endpoint to get attributes to write to geoJSON
state_count15 = requests.get('http://localhost:5000/state_count15')
state_metal15 = requests.get('http://localhost:5000/state_metal15')
state_issuer15 = requests.get('http://localhost:5000/state_issuer15')
state_type15 = requests.get('http://localhost:5000/state_type15')

ConnectionError: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /state_count15 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002220B869CC0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it',))

In [ ]:
#read the JSONs
state_count15 = state_count15.json()
state_metal15 = state_metal15.json()
state_issuer15 = state_issuer15.json()
state_type15 = state_type15.json()

In [351]:
#turn Count back into an int
for dicts in state_count15: 
    for keys in dicts: 
        dicts['Count'] = int(dicts['Count']) 
        
#turn Count back into an int
for dicts in state_type15: 
    for keys in dicts: 
        dicts['Count'] = int(dicts['Count']) 
        
#turn Count back into an int
for dicts in state_issuer15: 
    for keys in dicts: 
        dicts['Count'] = int(dicts['Count']) 

In [352]:
#lookup the states that we have data for count data to the geoJSON

counter1=0
counter2=0

while counter1 < len(state_count15):
    lookup_state = state_count15[counter1]['State']
    count_state = states['features'][counter2]['properties']['state_code']
    if lookup_state == count_state:
        states['features'][counter2]['properties']['state_count15'] = state_count15[counter1]['Count']
        counter2 = 0
        counter1 = counter1 +1
    else:
        counter2 = counter2 +1

In [353]:
#cast Count to int and break metal up into each type

source_var = state_metal15

year_data = pd.DataFrame(source_var)
year_data['Count'] = year_data['Count'].astype(int)
bronze15 = year_data.loc[year_data['Metal'] == 'Bronze'].reset_index(drop=True)
expanded_bronze15 = year_data.loc[year_data['Metal'] == 'Expanded Bronze'].reset_index(drop=True)
silver15=year_data.loc[year_data['Metal'] == 'Silver'].reset_index(drop=True)
gold15=year_data.loc[year_data['Metal'] == 'Gold'].reset_index(drop=True)
platinum15=year_data.loc[year_data['Metal'] == 'Platinum'].reset_index(drop=True)

In [354]:
#write metal data to geoJSON

counter1=0
counter2=0

pointer = bronze15
field = 'bronze_state15'

while counter1 < len(pointer):
    lookup_state = pointer['State'][counter1]
    count_state = states['features'][counter2]['properties']['state_code']
    if lookup_state == count_state:
        states['features'][counter2]['properties'][field] = pointer.loc[pointer['State'] == lookup_state]['Count'][counter1]
        counter2 = 0
        counter1 = counter1 +1
    else:
        counter2 = counter2 +1
        
counter1=0
counter2=0

pointer = expanded_bronze15
field = 'expanded_bronze_15'

while counter1 < len(pointer):
    lookup_state = pointer['State'][counter1]
    count_state = states['features'][counter2]['properties']['state_code']
    if lookup_state == count_state:
        states['features'][counter2]['properties'][field] = pointer.loc[pointer['State'] == lookup_state]['Count'][counter1]
        counter2 = 0
        counter1 = counter1 +1
    else:
        counter2 = counter2 +1

counter1=0
counter2=0
        
pointer = silver15
field = 'silver_state15'

while counter1 < len(pointer):
    lookup_state = pointer['State'][counter1]
    count_state = states['features'][counter2]['properties']['state_code']
    if lookup_state == count_state:
        states['features'][counter2]['properties'][field] = pointer.loc[pointer['State'] == lookup_state]['Count'][counter1]
        counter2 = 0
        counter1 = counter1 +1
    else:
        counter2 = counter2 +1

counter1=0
counter2=0        

pointer = gold15
field = 'gold_state15'

while counter1 < len(pointer):
    lookup_state = pointer['State'][counter1]
    count_state = states['features'][counter2]['properties']['state_code']
    if lookup_state == count_state:
        states['features'][counter2]['properties'][field] = pointer.loc[pointer['State'] == lookup_state]['Count'][counter1]
        counter2 = 0
        counter1 = counter1 +1
    else:
        counter2 = counter2 +1
        
counter1=0
counter2=0        

pointer = platinum15
field = 'platinum_state15'

while counter1 < len(pointer):
    lookup_state = pointer['State'][counter1]
    count_state = states['features'][counter2]['properties']['state_code']
    if lookup_state == count_state:
        states['features'][counter2]['properties'][field] = pointer.loc[pointer['State'] == lookup_state]['Count'][counter1]
        counter2 = 0
        counter1 = counter1 +1
    else:
        counter2 = counter2 +1

In [355]:
#write out issuers and the number of plans by each issuer ot the geoJSON
counter1 = 0
counter2 = 0
counter3 = 0
pointer = state_issuer15
year = 15
issuer = pointer[counter1]['Issuer']

while counter1 < len(state_issuer15):
    if pointer[counter1]['State'] == states['features'][counter2]['properties']['state_code']:
            states['features'][counter2]['properties'][f'issuer{year}_{counter3}'] = pointer[counter1]['Count']
            states['features'][counter2]['properties'][f'issuer{year}_{counter3}_name'] = pointer[counter1]['Issuer']
            state_reference = pointer[counter1]['State']
            if state_reference != states['features'][counter2]['properties']['state_code']:
                counter3 = 0
            counter2 = 0
            counter1 = counter1 + 1
            counter3 = counter3 + 1
            
    else:
        counter2 = counter2 + 1

In [356]:
#split out each plan type
type15_df = pd.DataFrame(state_type15)
EPO15 = type15_df.loc[type15_df['Type'] == 'EPO'].reset_index(drop=True)
PPO15 = type15_df.loc[type15_df['Type'] == 'PPO'].reset_index(drop=True)
HMO15 = type15_df.loc[type15_df['Type'] == 'HMO'].reset_index(drop=True)
POS15 = type15_df.loc[type15_df['Type'] == 'POS'].reset_index(drop=True)
Indemnity15 = type15_df.loc[type15_df['Type'] == 'Indemnity'].reset_index(drop=True)

In [357]:
counter1 = 0
counter2 = 0

pointer = EPO15
field = 'EPO15'

while counter1 < len(pointer):
    lookup_state = pointer['State'][counter1]
    count_state = states['features'][counter2]['properties']['state_code']
    if lookup_state == count_state:
        states['features'][counter2]['properties'][field] = pointer.loc[pointer['State'] == lookup_state]['Count'][counter1]
        counter2 = 0
        counter1 = counter1 +1
    else:
        counter2 = counter2 +1
        
counter1 = 0
counter2 = 0
        
pointer = PPO15
field = 'PPO15'

while counter1 < len(pointer):
    lookup_state = pointer['State'][counter1]
    count_state = states['features'][counter2]['properties']['state_code']
    if lookup_state == count_state:
        states['features'][counter2]['properties'][field] = pointer.loc[pointer['State'] == lookup_state]['Count'][counter1]
        counter2 = 0
        counter1 = counter1 +1
    else:
        counter2 = counter2 +1
  
counter1 = 0
counter2 = 0

pointer = HMO15
field = 'HMO15'

while counter1 < len(pointer):
    lookup_state = pointer['State'][counter1]
    count_state = states['features'][counter2]['properties']['state_code']
    if lookup_state == count_state:
        states['features'][counter2]['properties'][field] = pointer.loc[pointer['State'] == lookup_state]['Count'][counter1]
        counter2 = 0
        counter1 = counter1 +1
    else:
        counter2 = counter2 +1
        
counter1 = 0
counter2 = 0

pointer = POS15
field = 'POS15'

while counter1 < len(pointer):
    lookup_state = pointer['State'][counter1]
    count_state = states['features'][counter2]['properties']['state_code']
    if lookup_state == count_state:
        states['features'][counter2]['properties'][field] = pointer.loc[pointer['State'] == lookup_state]['Count'][counter1]
        counter2 = 0
        counter1 = counter1 +1
    else:
        counter2 = counter2 +1
        
counter1 = 0
counter2 = 0

pointer = Indemnity15
field = 'Indemnity15'

while counter1 < len(pointer):
    lookup_state = pointer['State'][counter1]
    count_state = states['features'][counter2]['properties']['state_code']
    if lookup_state == count_state:
        states['features'][counter2]['properties'][field] = pointer.loc[pointer['State'] == lookup_state]['Count'][counter1]
        counter2 = 0
        counter1 = counter1 +1
    else:
        counter2 = counter2 +1

In [358]:
states.to_json('./static/geoJson/modifiedStates.json')

In [359]:
states['features'][0]['properties']

{'name': 'Alabama',
 'density': 94.65,
 'state_code': 'AL',
 'state_count15': 402,
 'bronze_state15': 134,
 'silver_state15': 67,
 'gold_state15': 134,
 'platinum_state15': 67,
 'issuer15_19': 402,
 'issuer15_19_name': 'Blue Cross and Blue Shield of Alabama',
 'PPO15': 402}